## <CENTER> The $Z^0$ boson as an example
_____________________________________________

### Using what you have learned so far, let's see if we can find the $Z^0$ boson as an example!
_____________________________________________

In [ ]:
# import the ROOT library
import ROOT

### Get the data

In [ ]:
# http://opendata.atlas.cern/release/samples/2019/
# Look at the above web page, you will find simulated data and real experimental data.
# The data is divided into different final states, and for this exercise we'll look
# at dilepton events, so navigate into the "2lep" directory, then "data". There are several
# files, and we can load them all sequentially in a "chain". The tree we want to access
# inside the files is called "mini"
chains = {}
chains["data"] = ROOT.TChain("mini")
chains["mc2e"] = ROOT.TChain("mini")
chains["mc2mu"] = ROOT.TChain("mini")

# there are several files for the data
chains["data"].Add("http://opendata.atlas.cern/release/samples/2019/2lep/Data/data_A.2lep.root")
#chain.Add("http://opendata.atlas.cern/release/samples/2019/2lep/Data/data_B.2lep.root")
#chain.Add("http://opendata.atlas.cern/release/samples/2019/2lep/Data/data_C.2lep.root")
#chain.Add("http://opendata.atlas.cern/release/samples/2019/2lep/Data/data_D.2lep.root")

chains["mc2e"].Add("http://opendata.atlas.cern/release/samples/2019/2lep/MC/mc_361106.Zee.2lep.root")
chains["mc2mu"].Add("http://opendata.atlas.cern/release/samples/2019/2lep/MC/mc_361107.Zmumu.2lep.root")

for s in chains:
    print("Loaded sample %s containing %d events" % (s, chains[s].GetEntries()))

### Helper function for invariant mass

In [ ]:
# this function calculates the invariant mass of a two-particle system
def getInvMass(lep1_pt, lep1_eta, lep1_phi, lep2_pt, lep2_eta, lep2_phi):
    ''' Compute the invariant mass using the formula from the Introduction notebook (assume massless particles)'''
    import math
    m2 = 2*lep1_pt*lep2_pt*(math.cosh(lep1_eta-lep2_eta)-math.cos(lep1_phi-lep2_phi))
    return math.sqrt(m2) 

### Prepare histograms

In [ ]:
# create a histogram for invariant mass

# define the number of bins and axis range
nBins = 150
lowerEdge = 0
upperEdge = 150

# make a python dictionary holding the histograms for each of the samples
massHistos = {}
# also keep track of the number of ee and mumu pairs found in each sample
eePairs = {}
mumuPairs = {}

# loop over the samples to create the histograms and the counters
for sample in chains:
    massHistos[sample] = ROOT.TH1F("InvariantMass_%s" % sample, "; Invariant mass [GeV]; Number of events", nBins, lowerEdge, upperEdge)
    eePairs[sample] = 0
    mumuPairs[sample] = 0

### Run the event loop, find electron-positron/muon-antimuon pairs, evaluate the invariant mass, fill histogram(s). You are looking at data-not all events will contain a $Z^0$ boson. What cuts/requirements do you need to apply to filter out $Z^0$ boson candidate events? In the notebook 1 the different variables in the tree are listed.

In [ ]:
# clear the histogram, so you don't fill the same events several times if you re-run the loop
for s in massHistos:
    massHistos[s].Reset()

# loop over all of the samples, i.e. both data and the two simulated MC samples
for s in chains:
    # keep track of the number of events processed
    nEvents = 0
    maxEvents = 10000 # stop after this many events, useful during development
    # also keep track of the number of electron-positron and muon-antimuon pairs found
    for event in chains[s]:
        if nEvents > maxEvents:
            break
        # only process events which have exactly two leptons in them, as we expect for Z->ll
        if event.lep_n == 2:
            # skip events with different-flavor lepton pairs
            if event.lep_type[0] != event.lep_type[1]:
                continue
            # also skip events with same-sign leptons, we don't expect that from Z decays
            if event.lep_charge[0] == event.lep_charge[1]:
                continue
            m = getInvMass(event.lep_pt[0], event.lep_eta[0], event.lep_phi[0],
                          event.lep_pt[1], event.lep_eta[1], event.lep_phi[1])
            massHistos[s].Fill(m/1000) # divide by 1000 to go from MeV to GeV
            if nEvents % 1000 == 0: # only do a print-out every 1000 events
                print("Processed event %d of %s, filled m = %f" % (nEvents, s, m))
            # increment the counters depending on the lepton pair found
            if event.lep_type[0] == 11:
                eePairs[s] += 1
            elif event.lep_type[0] == 13:
                mumuPairs[s] += 1
        nEvents += 1

**Note:**: Since it takes a long time to run over all data, only run over a small number of events while developing. Then, when you are satisfied with the method, you can increase the number of events.

### Draw the invariant mass. Do you see the $Z^0$ boson peak?

Let's first have a look at the two MC samples, to see if there are any significant differences expected between electrons and muons

In [ ]:
# magic to get interactive plots
%jsroot on

# to set the y-axis range
yMax = 1.2*ROOT.TMath.Max(massHistos["mc2e"].GetMaximum(), massHistos["mc2mu"].GetMaximum())

canvas = ROOT.TCanvas("Invariant mass", "Invariant mass", 800, 600)
massHistos["mc2e"].SetLineColor(ROOT.kBlue)
massHistos["mc2e"].Draw()
massHistos["mc2e"].GetYaxis().SetRangeUser(0, yMax)
massHistos["mc2e"].Draw()
massHistos["mc2mu"].SetLineColor(ROOT.kRed)
massHistos["mc2mu"].Draw("SAME") # "SAME" is needed to not draw over the first histo

canvas.Draw()

for s in chains:
    if "mc" in s:
        print("Sample %s: found %d ee pairs and %d mumu pairs" % (s, eePairs[s], mumuPairs[s]))


### Make a fit to the data. What function should you use? Do we need a signal plus background model? Try different functions and ranges. Look at 2-Fitting-with-ROOT for help.

In [ ]:
%run hints/hint3.py

In [ ]:
### declare function
func = ROOT.TF1("...", ...)

In [ ]:
h_mass.Fit("fcn_name", "S", ...)
canvas.Draw()